<a href="https://colab.research.google.com/github/Insane110/Online-Info-generator-using-RAG-and-Llama-Index/blob/main/RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers #(if library not updated)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 71.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
!pip install llama-index llama-index-core python-dotenv googlesearch-python bs4 requests llama-index-llms-huggingface

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install transformers einops accelerate langchain bitsandbytes sentence_transformers langchain_community llama-index-embeddings-langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.0 MB/s eta 0:00:00


In [ ]:
!pip install opencv-python
!pip install pytesseract
!apt-get update
!apt-get install -y tesseract-ocr
!tesseract --version

In [ ]:
import googlesearch
from googlesearch import search
import requests
from bs4 import BeautifulSoup
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from langchain_community.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
import torch
import os
import re
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import pytesseract
from io import BytesIO
import urllib.parse
# load_dotenv()

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
def search_web(query, num_results):
    try:
        encoded_query = urllib.parse.quote(query)
        url = f"https://html.duckduckgo.com/html/?q={encoded_query}"

        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.find_all('a', class_='result__a')
        urls = []
        for result in results[:num_results]:
            href = result['href']
            # Extract the 'uddg' parameter from the href
            parsed_href = urllib.parse.urlparse(href)
            query_params = urllib.parse.parse_qs(parsed_href.query)
            if 'uddg' in query_params:
                # Decode the actual URL
                actual_url = query_params['uddg'][0]
                urls.append(actual_url)
            else:
                # If no 'uddg' parameter, just append the href
                urls.append(href)

        return urls
    except Exception as e:
        return []

In [ ]:
def text_from_image_table(image):
  img_resized = cv2.resize(image,
                          (int(image.shape[1] + (image.shape[1] * .35)),
                            int(image.shape[0] + (image.shape[0] * .5))),
                          interpolation=cv2.INTER_AREA)
  img_rgb = cv2.cvtColor(img_resized,cv2.COLOR_BGR2RGB)
  cv2_imshow()
  s = pytesseract.image_to_string(img_rgb)
  return str(s)

In [ ]:
def process_image_table(img_url):
    try:
        response = requests.get(img_url)
        response.raise_for_status()
        nparr = np.frombuffer(response.content, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        extracted_text = text_from_image_table(image)

        return extracted_text
    except Exception as e:
        return None

In [ ]:
def extract_table_data(table):
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all(['td', 'th'])
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)
    return pd.DataFrame(data)

In [ ]:
def scrape_url(urls):
    scraped_data = []
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract text from paragraphs
            paragraphs = soup.find_all('p')
            text_content = '\n'.join([para.text for para in paragraphs])

            # Extract and process images
            images = soup.find_all('img')
            image_content = []
            for img in images:
                img_url = img.get('src')
                if img_url:
                    if not img_url.startswith('http'):
                        img_url = f"{url}/{img_url}"
                    img_content = process_image_table(img_url)
                    if img_content:
                        image_content.append(img_content)

            # Extract table data
            tables = soup.find_all('table')
            table_content = []
            for table in tables:
                df = extract_table_data(table)
                table_content.append(df.to_string(index=False))

            combined_content = (f"Text Content:\n{text_content}\n\n"
                                f"Extracted Image Content:\n{' '.join(image_content)}\n\n"
                                f"Extracted Table Content:\n{' '.join(table_content)}")
            scraped_data.append(combined_content)
        except requests.exceptions.RequestException as e:
            pass
    return scraped_data

In [ ]:
def docs(query,num_results):
  urls = search_web(query,num_results)
  text = scrape_url(urls)
  documents = [Document(text=t) for t in text]

  return documents

In [ ]:
def process_documents(documents):
    non_empty_documents = []
    for doc in documents:
        if hasattr(doc, 'text') and doc.text.strip():
            non_empty_documents.append(doc)
        else:
            print(f"Skipping empty document: {doc}")
    return non_empty_documents

In [ ]:
system_prompt="""
you are an educational bot that gives info in a detailed manner with respect to the question asked.
Your answers are in a chronological order.
you are to first give brief introduction about the question asked and then explain every aspect of the requested answer in a detailed manner.
"""

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [ ]:
llm = HuggingFaceLLM(
    context_window=3072,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.5, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16,"quantization_config": quantization_config}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
!pip install llama-index-vector-stores-faiss

In [ ]:
!pip install faiss-gpu #for gpu

In [ ]:
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.storage.storage_context import StorageContext
import faiss

In [ ]:
lc_embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
  )
embed_model = LangchainEmbedding(lc_embed_model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-24-e0d57d63d7c4>:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
service_context = ServiceContext.from_defaults(
  chunk_size=3072,
  llm=llm,
  embed_model=embed_model
)

<ipython-input-34-cf3f27f2b05b>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
def process_query(search_query, num_results, user_query):
    urls = search_web(search_query, num_results)
    scraped_texts = scrape_url(urls)
    documents = [Document(text=text) for text in scraped_texts]
    # dimensions of sentence transformers
    d = 384
    faiss_index = faiss.IndexFlatL2(d)
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
            documents,
            storage_context=storage_context,
            service_context=service_context)
    query_engine = index.as_query_engine(llm=llm,embed_model=embed_model)
    response = query_engine.query(user_query)

    return str(response)

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.0
    Uninstalling tomlkit-0.13.0:
      Successfully uninstalled tomlkit-0.13.0


In [ ]:
import gradio as gr
iface = gr.Interface(
    fn=process_query,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your search query (e.g., climate change effects)...", label="Search Query"),
        gr.Slider(minimum=1, maximum=10, step=1, value=5, label="Number of Search Results", info="Select the number of search results to fetch"),
        gr.Textbox(lines=4, placeholder="Enter your question about the search results (e.g., How does climate change affect biodiversity?)...", label="Your Question")
    ],
    outputs="text",
    title="Web Search and Query Engine",
    description="This tool helps you fetch web content based on your search query and allows you to ask specific questions about the results. Simply enter your search query, select the number of results you want, and then ask your question.",
    theme="default",
    allow_flagging="never",
    live=False
)



In [ ]:
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c3de531d8c585ff6c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
